In [1]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"


import findspark
findspark.init()
findspark.find()

import pyspark

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 229 kB in 3s (88.3 kB/s)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
45 packages can be upgraded. Run 'apt list --upgradable' to see them.
tar: spark-3.2.1-bin-hadoop3.2.tgz: Cannot open

In [2]:
import pyspark
import pyspark.sql.functions as F
import numpy as np
from pyspark.sql import SparkSession
import numpy as np
from datetime import datetime, timedelta
from pyspark.sql.window import Window
import numpy as np
import math
import pandas as pd
import subprocess
from google.colab import drive
import plotly.express as px
import numpy as np
import pandas as pd


In [3]:
################################
appName = 'AntonioLorenzoFinalProject'
spark = SparkSession.builder\
                    .appName(appName) \
                    .getOrCreate()
sc = spark.sparkContext

In [4]:
spark

In [5]:
# load the data from the csv file
filename='uber.csv'
df = spark.read.csv(filename, header=True, inferSchema=True)
df = df.orderBy("date")
df.show()

+-------+------+----------+------------------+------------------+------------------+------------------+------------------+------------+-----+
|    cik|ticker|      date|              open|              high|               low|             close|          adjclose|      volume|   rn|
+-------+------+----------+------------------+------------------+------------------+------------------+------------------+------------+-----+
|1543151|  UBER|2019-05-10|              42.0|              45.0|41.060001373291016| 41.56999969482422| 41.56999969482422|1.86322496E8|79688|
|1543151|  UBER|2019-05-13|38.790000915527344|  39.2400016784668| 36.08000183105469|37.099998474121094|37.099998474121094|   7.94424E7|79687|
|1543151|  UBER|2019-05-14|38.310001373291016|39.959999084472656|36.849998474121094|39.959999084472656|39.959999084472656|   4.66611E7|79686|
|1543151|  UBER|2019-05-15|39.369998931884766|41.880001068115234| 38.95000076293945|41.290000915527344|41.290000915527344|   3.60861E7|79685|
|15431

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.sql.functions import year, month, dayofmonth

# Extract year, month, and day from the date column
data_with_features = df.withColumn("year", year("date")) \
                                  .withColumn("month", month("date")) \
                                  .withColumn("day", dayofmonth("date"))
data_with_features = data_with_features.orderBy("date")

# Calculate the index to make the split (80% train 20% test)
split_index = int(0.8 * data_with_features.count())

# Split the data into train and test sets
train_data = data_with_features.limit(split_index)
test_data = data_with_features.subtract(train_data)

# Define the features column
feature_columns = ['year', 'month', 'day']
vector_assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")

# Define the DecisionTreeRegressor
dtr = DecisionTreeRegressor(featuresCol="features", labelCol="close")

# Create a pipeline
pipeline = Pipeline(stages=[vector_assembler, dtr])

# Define parameter grid for hyperparameter tuning
param_grid = ParamGridBuilder() \
    .addGrid(dtr.maxDepth, [3, 5, 7]) \
    .addGrid(dtr.maxBins, [3,5, 10]) \
    .build()

# Define evaluator
evaluator = RegressionEvaluator(labelCol="close", predictionCol="prediction", metricName="rmse")

# Create CrossValidator
cross_val = CrossValidator(estimator=pipeline,
                           estimatorParamMaps=param_grid,
                           evaluator=evaluator,
                           numFolds=10,
                           seed=42)      # Set seed for reproducibility

# Fit the model
cv_model = cross_val.fit(train_data)

# Get best model
best_model = cv_model.bestModel

# Make predictions on the test set
predictions = best_model.transform(test_data)

# Evaluate the model on test data
rmse = evaluator.evaluate(predictions)

print("Root Mean Squared Error (RMSE) on test data =", rmse)


In [ ]:
params = [{p.name: v for p, v in m.items()} for m in cv_model.getEstimatorParamMaps()]
import pandas as pd

pd.DataFrame.from_dict([
    {cv_model.getEvaluator().getMetricName(): metric, **ps}
    for ps, metric in zip(params, cv_model.avgMetrics)
])

In [ ]:
import time
# Split data into train and test sets
split_index = int(data_with_features.count() - 7)

# Split the data into train and test sets
train_data = data_with_features.limit(split_index)
test_data = data_with_features.subtract(train_data)

# Define the features column
feature_columns = ['year', 'month', 'day']
vector_assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")

# Define the model with the best cv values
best_dtr = DecisionTreeRegressor(featuresCol="features", labelCol="close", maxDepth=7, maxBins= 10)

# Create a pipeline
pipeline = Pipeline(stages=[vector_assembler, best_dtr])

# Fit the model on the entire dataset except the last 7 days
start_time = time.time()
model = pipeline.fit(train_data)
end_time = time.time()
print("Training time: {:.2f} seconds".format(end_time - start_time))

# Make predictions on the entire dataset
predictions = model.transform(test_data)

# Define evaluator
evaluator = RegressionEvaluator(labelCol="close", predictionCol="prediction", metricName="rmse")
evaluator_mae = RegressionEvaluator(labelCol="close", predictionCol="prediction", metricName="mae")
evaluator_r2 = RegressionEvaluator(labelCol="close", predictionCol="prediction", metricName="r2")

# Evaluate the model on test data
rmse = evaluator.evaluate(predictions)
mae = evaluator_mae.evaluate(predictions)
r2 = evaluator_r2.evaluate(predictions)

print("Model results for last 7 days")
print("Root Mean Squared Error (RMSE) on test data =", rmse)
print("Mean Absolute Error (MAE) on test data =", mae)
print("R-squared (R2) on test data =", r2)

In [ ]:
predictions.show()

In [ ]:
test_data = test_data.orderBy("date")
test_data.show()

In [ ]:
import matplotlib.pyplot as plt
import random

predictions = predictions.orderBy("date")

# Collecting data from DataFrame columns as lists
dates = predictions.select("date").rdd.flatMap(lambda x: x).collect()
predictions_values = predictions.select("prediction").rdd.flatMap(lambda x: x).collect()
real_values = predictions.select("close").rdd.flatMap(lambda x: x).collect()
max_val = max(real_values) + 5
min_val = min(real_values) - 5
valores_random = [random.uniform(min_val, max_val) for _ in range(7)]

# Plotting the data
plt.plot(dates, predictions_values, label='Predicciones', linestyle='-', marker='o', color='b')
plt.plot(dates, real_values, label='Real', linestyle='--', marker='s', color='r')
plt.plot(dates, valores_random, label='Random', linestyle=':', marker='s', color='g')

# Labels and legend
plt.xlabel('Fecha')
plt.ylabel('Valor')
plt.title('Predicciones vs Real')
plt.legend()

# Displaying the plot
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()